In [ ]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 8.7 MB/s eta 0:00:00


In [ ]:
# Necessary packages
import pathlib
import textwrap
import json

import google.generativeai as genai
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from IPython.display import display
from IPython.display import Markdown
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from google.colab import userdata

In [ ]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-001
models/gemini-1.5-flash-latest
models/gemini-1.5-pro
models/gemini-1.5-pro-001
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')

In [ ]:
# Muat file Excel
df = pd.read_excel('/content/Sentimen_2.xlsx')

In [ ]:
df.head(2)

,Sentimen,True_Label
0,Kebijakan Pariwisata Halal harus didukung kare...,2
1,Kebijakan wisata halal akan mengakomodir semua...,2


In [ ]:
df['pred_label'] = ''

In [ ]:
df.head(2)

,Sentimen,True_Label,pred_label
0,Kebijakan Pariwisata Halal harus didukung kare...,2,
1,Kebijakan wisata halal akan mengakomodir semua...,2,


In [ ]:
# Convert the DataFrame to JSON using the to_json() method

json_data = df[['Sentimen','pred_label']].to_json(orient='records')

# Print the JSON data
print(json_data)

[{"Sentimen":"Kebijakan Pariwisata Halal harus didukung karena dapat meningkatkan kenyamanan wisatawan Muslim, memperluas pasar pariwisata, dan mendukung perkembangan ekonomi lokal dengan menarik lebih banyak wisatawan dari negara-negara berpenduduk mayoritas Muslim.","pred_label":""},{"Sentimen":"Kebijakan wisata halal akan mengakomodir semua target wisatawan sehingga bisa meningkatkan devisa kas pemerintahan. Kebijakan pariwisata halal juga bisa meningkatkan wisatawan muslim untuk lebih menjajal pariwisata di indonesia tanpa harus khawatir melakukan pelanggaran terhadap syariat agama","pred_label":""},{"Sentimen":"Klau buat kenyamanan wisatawan muslim sya mndukung","pred_label":""},{"Sentimen":"Lebih mengutamakan agama dalam berwisata tapi tetap berwisata dengan nyaman","pred_label":""},{"Sentimen":"lebih mudah  dalam berwisata, makanan halal mudah tersedia","pred_label":""},{"Sentimen":"Mayoritas masyarakat Desa Lamajang beragama. Di dalam agama manapun ada konsep halal hanya saj pe

In [ ]:
prompt = f"""
Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
{json_data}
```
"""

print(prompt)


Anda adalah ahli bahasa yang ahli dalam mengklasifikasikan sentimen opini masyarakat ke dalam label Positif, Netral atau Negatif.
Bantu saya mengklasifikasikan opini masyarakat terhadap kebijakan pariwisata halal menjadi: Positif (label= 2) Netral (label = 0)dan Negatif (label = 1).
Dalam keluaran Anda, hanya kembalikan kode Json kembali sebagai keluaran - yang disediakan di antara tiga kutu balik.
Tugas Anda adalah memperbarui label yang diprediksi di bawah 'pred_label' dalam kode Json.
Jangan membuat perubahan apapun pada format kode Json.

```
[{"Sentimen":"Kebijakan Pariwisata Halal harus didukung karena dapat meningkatkan kenyamanan wisatawan Muslim, memperluas pasar pariwisata, dan mendukung perkembangan ekonomi lokal dengan menarik lebih banyak wisatawan dari negara-negara berpenduduk mayoritas Muslim.","pred_label":""},{"Sentimen":"Kebijakan wisata halal akan mengakomodir semua target wisatawan sehingga bisa meningkatkan devisa kas pemerintahan. Kebijakan pariwisata halal juga

In [ ]:
response = model.generate_content(prompt)

print(response.text)

```
[{"Sentimen":"Kebijakan Pariwisata Halal harus didukung karena dapat meningkatkan kenyamanan wisatawan Muslim, memperluas pasar pariwisata, dan mendukung perkembangan ekonomi lokal dengan menarik lebih banyak wisatawan dari negara-negara berpenduduk mayoritas Muslim.","pred_label":2},{"Sentimen":"Kebijakan wisata halal akan mengakomodir semua target wisatawan sehingga bisa meningkatkan devisa kas pemerintahan. Kebijakan pariwisata halal juga bisa meningkatkan wisatawan muslim untuk lebih menjajal pariwisata di indonesia tanpa harus khawatir melakukan pelanggaran terhadap syariat agama","pred_label":2},{"Sentimen":"Klau buat kenyamanan wisatawan muslim sya mndukung","pred_label":2},{"Sentimen":"Lebih mengutamakan agama dalam berwisata tapi tetap berwisata dengan nyaman","pred_label":2},{"Sentimen":"lebih mudah  dalam berwisata, makanan halal mudah tersedia","pred_label":2},{"Sentimen":"Mayoritas masyarakat Desa Lamajang beragama. Di dalam agama manapun ada konsep halal hanya saj per

In [ ]:
# Clean the data by stripping the backticks
json_data = response.text.strip("`")

# Load the cleaned data and convert to DataFrame
data = json.loads(json_data)
df_sample = pd.DataFrame(data)

df_sample

,Sentimen,pred_label
0,Kebijakan Pariwisata Halal harus didukung kare...,2
1,Kebijakan wisata halal akan mengakomodir semua...,2
2,Klau buat kenyamanan wisatawan muslim sya mndu...,2
3,Lebih mengutamakan agama dalam berwisata tapi ...,2
4,"lebih mudah dalam berwisata, makanan halal mu...",2
5,Mayoritas masyarakat Desa Lamajang beragama. D...,2
6,melihat sejauh ini mungkin nilai nilai spiritu...,2
7,memaksimalkan pelayanan kepada wisatawan,2
8,Memberi kenyamanan pada wisatawan muslim,2
9,memberi kenyamanan pengunjung tidak melihat ya...,2


In [ ]:
# prompt: Overwrite pred_label from 'df' into pred_label in 'train_set_sample'

df['pred_label'] = df_sample['pred_label'].values
df

,Sentimen,True_Label,pred_label
0,Kebijakan Pariwisata Halal harus didukung kare...,2,2
1,Kebijakan wisata halal akan mengakomodir semua...,2,2
2,Klau buat kenyamanan wisatawan muslim sya mndu...,2,2
3,Lebih mengutamakan agama dalam berwisata tapi ...,2,2
4,"lebih mudah dalam berwisata, makanan halal mu...",2,2
5,Mayoritas masyarakat Desa Lamajang beragama. D...,2,2
6,melihat sejauh ini mungkin nilai nilai spiritu...,2,2
7,memaksimalkan pelayanan kepada wisatawan,2,2
8,Memberi kenyamanan pada wisatawan muslim,2,2
9,memberi kenyamanan pengunjung tidak melihat ya...,2,2


In [ ]:
# prompt: Menggunakan df DataFrame: saya ingin menyimpan file kedalam bentuk excel

df.to_excel('data 2.xlsx')